# Исследование надежности заемщиков

## Общая информация

**Заказчик — кредитный отдел банка.** 

**Входные данные от банка**: статистика о платёжеспособности клиентов.

Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.
**Заказчик — кредитный отдел банка.** 

**Цель исследования**: определить наличие влияния параметров клиента на факт погашения кредита в срок. 

**Вопросы заказчика**:

- Влияет ли семейное положение клиента на факт погашения кредита в срок?
- Влияет ли количество детей клиента на факт погашения кредита в срок?

## Чтение данных

In [1]:
import pandas as pd
import numpy as np

<div class="alert alert-block alert-info">
<b>Tip:</b> Если вы запускаете Jupyter Notebook на локальном ПК, можно использовать блок `try...except`, чтобы при проверке вашего проекта в среде Яндекса, у ревьюера не возникала ошибка, если вы вдруг забыли изменить путь к файлу :) </div>

In [2]:
try:
    df = pd.read_csv('/datasets/data.csv')
except:
    df = pd.read_csv('data.csv')

In [3]:
# Понадобится позже
copy = df.copy()

### Изучаем на данные

#### Описание полей данных (полезно иметь в быстром доступе)

- *children* — количество детей в семье
- *days_employed* — общий трудовой стаж в днях
- *dob_years* — возраст клиента в годах
- *education* — уровень образования клиента
- *education_id* — идентификатор уровня образования
- *family_status* — семейное положение
- *family_status_id* — идентификатор семейного положения
- *gender* — пол клиента
- *income_type* — тип занятости
- *debt* — имел ли задолженность по возврату кредитов
- *total_income* — ежемесячный доход
- *purpose* — цель получения кредита

In [4]:
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


In [5]:
# 5 случайных строк датафрейма
df.sample(5)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
14361,0,-1309.319009,57,среднее,1,гражданский брак,1,F,сотрудник,0,173685.130550,сыграть свадьбу
7661,0,341279.567523,65,среднее,1,гражданский брак,1,F,пенсионер,0,50405.217514,на покупку своего автомобиля
17517,1,-506.088160,53,высшее,0,женат / замужем,0,M,компаньон,1,135607.236400,строительство собственной недвижимости
4801,0,-2111.478602,29,ВЫСШЕЕ,0,Не женат / не замужем,4,F,сотрудник,0,270468.966016,профильное образование
14759,0,-319.313960,44,среднее,1,Не женат / не замужем,4,F,сотрудник,0,161126.152068,дополнительное образование


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Пропуски в полях:
- *days_employed*
- *total_income*

## Дубликаты


In [7]:
df.duplicated().sum()

54

In [8]:
# keep=False -- by setting keep on False, all duplicates are True.

df[df.duplicated(keep=False)].sort_values('dob_years').head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
19321,0,NaN,23,среднее,1,Не женат / не замужем,4,F,сотрудник,0,NaN,сделка с подержанным автомобилем
15892,0,NaN,23,среднее,1,Не женат / не замужем,4,F,сотрудник,0,NaN,сделка с подержанным автомобилем
18328,0,NaN,29,высшее,0,женат / замужем,0,M,сотрудник,0,NaN,покупка жилой недвижимости
3452,0,NaN,29,высшее,0,женат / замужем,0,M,сотрудник,0,NaN,покупка жилой недвижимости
8629,1,NaN,30,высшее,0,женат / замужем,0,F,сотрудник,0,NaN,покупка коммерческой недвижимости
21281,1,NaN,30,высшее,0,женат / замужем,0,F,сотрудник,0,NaN,покупка коммерческой недвижимости
680,1,NaN,30,высшее,0,женат / замужем,0,F,госслужащий,0,NaN,покупка жилья для семьи
18349,1,NaN,30,высшее,0,женат / замужем,0,F,госслужащий,0,NaN,покупка жилья для семьи
8490,1,NaN,31,среднее,1,женат / замужем,0,F,компаньон,0,NaN,покупка жилья
13878,1,NaN,31,среднее,1,женат / замужем,0,F,компаньон,0,NaN,покупка жилья


**Из опыта: сначала обрабатываем дубликаты, затем делаем остальную предобработку.**

- Удаляем / оставляем? -- пояснение
- В реальных задачах: общаемся с ответственным подразделением + с заказчиком

In [9]:
# df.drop_duplicates(inplace=True)

# Для примера: чтобы не менять исходный датафрейм
copy.drop_duplicates(inplace=True)

## Аномалии

#### Вопросы от вас

<div class="alert alert-block alert-info">
<b>Вопрос:</b> Можно ли разобрать варианты работы с артефактами?</div>

<div class="alert alert-block alert-info">
<b>Вопрос:</b> присоединяюсь к предыдущему вопросу, не совсем понятно, как разобраться, когда нужно серьёзно разбираться с чем-то, а когда достаточно привести к нормальному виду для вычислений, как это происходит в работе, какие пути избавления от аномалий? От чего отталкиваться? Спасибо!</div>

In [10]:
for col in df.columns:
    val_count = pd.DataFrame()
    val_count[col] = df[col].value_counts()
    val_count[col + ' %'] = round(df[col].value_counts(normalize=True) * 100, 2)
    display(val_count)

,children,children %
0,14149,65.73
1,4818,22.38
2,2055,9.55
3,330,1.53
20,76,0.35
-1,47,0.22
4,41,0.19
5,9,0.04


,days_employed,days_employed %
-327.685916,1,0.01
-1580.622577,1,0.01
-4122.460569,1,0.01
-2828.237691,1,0.01
-2636.090517,1,0.01
...,...,...
-7120.517564,1,0.01
-2146.884040,1,0.01
-881.454684,1,0.01
-794.666350,1,0.01


,dob_years,dob_years %
35,617,2.87
40,609,2.83
41,607,2.82
34,603,2.80
38,598,2.78
42,597,2.77
33,581,2.70
39,573,2.66
31,560,2.60
36,555,2.58


,education,education %
среднее,13750,63.88
высшее,4718,21.92
СРЕДНЕЕ,772,3.59
Среднее,711,3.30
неоконченное высшее,668,3.10
ВЫСШЕЕ,274,1.27
Высшее,268,1.25
начальное,250,1.16
Неоконченное высшее,47,0.22
НЕОКОНЧЕННОЕ ВЫСШЕЕ,29,0.13


,education_id,education_id %
1,15233,70.77
0,5260,24.44
2,744,3.46
3,282,1.31
4,6,0.03


...

,gender,gender %
F,14236,66.14
M,7288,33.86
XNA,1,0.00


,income_type,income_type %
сотрудник,11119,51.66
компаньон,5085,23.62
пенсионер,3856,17.91
госслужащий,1459,6.78
безработный,2,0.01
предприниматель,2,0.01
в декрете,1,0.00
студент,1,0.00


,debt,debt %
0,19784,91.91
1,1741,8.09


,total_income,total_income %
169846.427535,1,0.01
257737.077768,1,0.01
200508.675866,1,0.01
106196.235958,1,0.01
248730.171354,1,0.01
...,...,...
175057.266090,1,0.01
101516.604975,1,0.01
239154.168013,1,0.01
165009.733021,1,0.01


,purpose,purpose %
свадьба,797,3.70
на проведение свадьбы,777,3.61
сыграть свадьбу,774,3.60
операции с недвижимостью,676,3.14
покупка коммерческой недвижимости,664,3.08
операции с жильем,653,3.03
покупка жилья для сдачи,653,3.03
операции с коммерческой недвижимостью,651,3.02
покупка жилья,647,3.01
жилье,647,3.01


### Количество детей (*children*)

- -1? -- Замена по модулю
- 20? -- Замена на `2`

Информируем соответствующее подразделение + заказчика

In [11]:
df['children'] = df['children'].abs()

### Общий трудовой стаж в днях (*days_employed*)

In [12]:
df['days_employed'].describe()

count     19351.000000
mean      63046.497661
std      140827.311974
min      -18388.949901
25%       -2747.423625
50%       -1203.369529
75%        -291.095954
max      401755.400475
Name: days_employed, dtype: float64

In [13]:
df['days_employed'][df['days_employed'] < 0].count()

15906

In [14]:
df['days_employed'][df['days_employed'] > 18000].count()

3445

- Большие значения: в днях? Часы? -- заменяем значениями функции: например, рассчитываем стаж от начала трудового возраста (18 лет) до текущего возраста клиента
- Отрицательные значения: техническая ошибка? (информируем соответствующее подразделение + заказчика) -> заменяем на положительные (`abs()`)

## Возвращаемся к пропускам

### Количество пропущенных значений

In [15]:
df.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

### Доля пропущенных значений

In [16]:
round(df.isna().sum() * 100 / len(df), 2)

children             0.0
days_employed       10.1
dob_years            0.0
education            0.0
education_id         0.0
family_status        0.0
family_status_id     0.0
gender               0.0
income_type          0.0
debt                 0.0
total_income        10.1
purpose              0.0
dtype: float64

#### Заполнение пропущенных значений

Вопросы заказчика?

- Влияет ли **семейное положение** клиента на факт погашения кредита в срок?
- Влияет ли **количество детей** клиента на факт погашения кредита в срок?

#### Вопросы от вас

<div class="alert alert-block alert-info">
<b>Вопрос:</b> Есть вопрос. Вот нам рассказывали, что сначала мы избавляемся от нулевых значений, потом занимаемся дубликатами и артефактами с аномалиями, затем уже переходим к исследованию гипотез. В моем текущем проекте при отработке нулевых значений одного из столбцов возник вопрос чем их заполнять. Я решил провести мини исследование на эту тему. Для этого пришлось избавиться от аномалий и артефактов во всех остальных столбцах (иначе невозможно было получить нужные данные). А когда заполнил пустые значения и перешёл к этапу обработки дубликатов и артефактов, выяснилось, что я их уже все исправил когда занимался пропусками. Как это лучше всего оформить, чтобы было понятно и ревьюеру, и я потом сам не заблудился в своем объёмном проекте?</div>

<div class="alert alert-block alert-info">
<b>Вопрос:</b> Наталья, привет! Подскажи, пожалуйста, какой смысл переводить количественные значения NaN определенного столбца данных в медиану этого диапазона? Не вижу разницы, если их просто выкинуть из исследования (или разница есть и неочевидная, тогда какое есть объяснение?). Да и функции по арифметическим операциям работают с NaN, по крайней мере те, что я успевал использовать. Где тут собака зарыта?) </div>

### Ежемесячный доход (*total_income*)

Есть ли зависимость от других полей в данных?

In [17]:
df[df['total_income'].isna()].describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,2174.000000,0.0,2174.000000,2174.000000,2174.000000,2174.000000,0.0
mean,0.555198,NaN,43.632015,0.800828,0.975161,0.078197,NaN
std,1.468315,NaN,12.531481,0.530157,1.418220,0.268543,NaN
min,0.000000,NaN,0.000000,0.000000,0.000000,0.000000,NaN
25%,0.000000,NaN,34.000000,0.250000,0.000000,0.000000,NaN
50%,0.000000,NaN,43.000000,1.000000,0.000000,0.000000,NaN
75%,1.000000,NaN,54.000000,1.000000,1.000000,0.000000,NaN
max,20.000000,NaN,73.000000,3.000000,4.000000,1.000000,NaN


- *days_employed*?

In [18]:
# df[df['total_income'].isna()]

#### Заполнение пропусков:
- В задании: заполнить медианным значением
- В реальной жизни: заполняем на основании факторов или совокупности факторов (возраст, пол, уровень образования, источник дохода)
- Также в реальной жизни: не обязательно стараться заполнить все пропуски (смотрим на вопросы заказчика. Если наличие пропусков в столбце не влияет на переменные, которые интересны заказчику -- не обязательно заполнять их)

In [19]:
# По заданию:

# df['total_income'].fillna((df['total_income'].median()), inplace=True)

In [20]:
# Можно посмотреть на различные значения (min, max, mean, median) в группировке по income_type

df.groupby('income_type').agg(
    min_income=('total_income', 'min'),
    max_income=('total_income', 'max'),
    mean_income=('total_income', 'mean'),
    median_income=('total_income', 'median')
)

,min_income,max_income,mean_income,median_income
income_type,,,,
безработный,59956.991984,2.027225e+05,131339.751676,131339.751676
в декрете,53829.130729,5.382913e+04,53829.130729,53829.130729
госслужащий,29200.077193,9.104515e+05,170898.309923,150447.935283
компаньон,28702.812889,2.265604e+06,202417.461462,172357.950966
пенсионер,20667.263793,7.351033e+05,137127.465690,118514.486412
предприниматель,499163.144947,4.991631e+05,499163.144947,499163.144947
сотрудник,21367.648356,1.726276e+06,161380.260488,142594.396847
студент,98201.625314,9.820163e+04,98201.625314,98201.625314


In [21]:
# Заполняем медианой в группировке по income_type

df['total_income_new'] = df.groupby(['income_type'])['total_income'].apply(lambda x: x.fillna(x.median()))

# Альтернативный способ:
# df['total_income'].fillna(df.groupby('income_type')['total_income'].transform('median')) 

### Общий трудовой стаж в днях (*days_employed*)

In [22]:
df.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
total_income_new       0
dtype: int64

In [23]:
df['days_employed'].describe()

count     19351.000000
mean      63046.497661
std      140827.311974
min      -18388.949901
25%       -2747.423625
50%       -1203.369529
75%        -291.095954
max      401755.400475
Name: days_employed, dtype: float64

#### Заполнение пропусков

- Заполняем на основании факторов или совокупности факторов (возраст, пол) -> медиана

## Создание дополнительных столбцов на основании значений

In [24]:
def income_level(income):
    if income <= 30000:
        return 'E'
    elif (income >= 30001) and (income <= 50000):
        return 'D'
    elif (income >= 50001) and (income <= 200000):
        return 'C'
    elif (income >= 200001) and (income <= 1000000):
        return 'B'
    else:
        return 'A'

In [25]:
df['total_income_category_1'] = df['total_income_new'].apply(income_level)

In [26]:
# Альтернативный способ

col = 'total_income_new'
conditions = [df[col] <= 30000,
              (df[col] >= 30001) & (df[col] <= 50000),
              (df[col] >= 50001) & (df[col] <= 200000),
              (df[col] >= 200001) & (df[col] <= 1000000)]

choices = ['E', 'D', 'C', 'B']
    
df['total_income_category_2'] = np.select(conditions, choices, default='A') # default = else

In [27]:
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,total_income_new,total_income_category_1,total_income_category_2
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья,253875.639453,B,B
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля,112080.014102,C,C
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья,145885.952297,C,C
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование,267628.550329,B,B
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу,158616.077870,C,C


## Обработка поля *Цель получения кредита* (*purpose*)

In [28]:
df['purpose'].value_counts()

свадьба                                   797
на проведение свадьбы                     777
сыграть свадьбу                           774
операции с недвижимостью                  676
покупка коммерческой недвижимости         664
операции с жильем                         653
покупка жилья для сдачи                   653
операции с коммерческой недвижимостью     651
покупка жилья                             647
жилье                                     647
покупка жилья для семьи                   641
строительство собственной недвижимости    635
недвижимость                              634
операции со своей недвижимостью           630
строительство жилой недвижимости          626
покупка недвижимости                      624
покупка своего жилья                      620
строительство недвижимости                620
ремонт жилью                              612
покупка жилой недвижимости                607
на покупку своего автомобиля              505
заняться высшим образованием      

Подробнее про [Mystem](https://yandex.ru/dev/mystem/)

In [29]:
from pymystem3 import Mystem
from collections import Counter

ModuleNotFoundError: No module named 'pymystem3'

In [ ]:
ms = Mystem()

data = df['purpose'].unique().tolist()

In [ ]:
lemmas = []

for el in data:
    lemmas += ms.lemmatize(el) # лемматизация

print(Counter(lemmas))

In [ ]:
purposes = [('авто', 'операции с автомобилем'), 
            ('недвиж', 'операции с недвижимостью'),
            ('жил', 'операции с недвижимостью'),
            ('коммерческ', 'операции с недвижимостью'),
            ('свадьб', 'проведение свадьбы'),
            ('образован', 'получение образования')
           ]

for p in purposes:
    df.loc[df['purpose'].str.contains(p[0]), 'purpose_type'] = p[1]

print(df.purpose_type.value_counts())

In [ ]:
# Или проще:

df.loc[df['purpose'].str.contains('свадьб'), 'purpose_type'] = 'проведение свадьбы'
df.head()

## Ответы на вопросы заказчика

- Есть ли зависимость между количеством детей и возвратом кредита в срок?
- Есть ли зависимость между семейным положением и возвратом кредита в срок?
- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

*debt* — имел ли задолженность по возврату кредитов

### Есть ли зависимость между [параметр] и возвратом кредита в срок?

In [ ]:
# % возвратов НЕ в срок

def correlation(col):
    pivot = df.pivot_table(index=col, columns='debt', aggfunc='size')
    pivot['%'] = round(pivot[1] / (pivot[1] + pivot[0]) * 100, 2)
    return pivot

In [ ]:
correlation('total_income_category_1')

### Как разные цели кредита (*purpose_type*) влияют на его возврат в срок?

In [ ]:
correlation('purpose_type')

## Общие выводы и рекомендации 

### Выводы
- Тезисные выводы по всем разделам
    - Пропуски / дубликаты / аномалии
    - Как обработали?
    
- Выводы по возвращаемости кредитов в сроки и взаимосвязи с параметрами клиента
    - Количество детей
    - Семейное положение
    - Уровень дохода
    - Цель кредита

### Рекомендации
- Сформулировать вопросы для заказчика и ответственных подразделений:
    - повышение качества данных (аномалии / пропуски / дубликаты)
- Расширить запросы заказчика:
    - на что еще интересно было бы посмотреть?
    - рекомендации по итогам анализа.